# GPU Computing with Julia

In [ ]:
gethostname()

## Topology of a GPU

<img src="../imgs/gpu_topology.png" width=1300px>

**Source:** [Sivalingam, Karthee. "GPU Acceleration of a Theoretical Particle Physics Application." Master's Thesis, The University of Edinburgh (2010).](https://static.epcc.ed.ac.uk/dissertations/hpc-msc/2009-2010/Karthee%20Sivalingam.pdf)

* **SM** = Streaming Multiprocessor
* **SP** = Streaming Processor

### NVIDIA A100 SXM4

<img src="../imgs/a100_front.png" width=800px>

<img src="../imgs/a100_SM.png" width=400px>

**Source:** [NVIDIA whitepaper](https://images.nvidia.com/aem-dam/en-zz/Solutions/data-center/nvidia-ampere-architecture-whitepaper.pdf)

| Kind                       | Count            |
|----------------------------|------------------|
| **SMs**                    | 108              |
| **CUDA cores** / FP32 ALUs | 6912 (64 per SM) |
| **Tensor cores**           | 432 (4 per SM)   |

* **ALU** = Arithmetic Logical Unit


In [ ]:
# using GPUInspector
# gpuinfo()

## JuliaGPU

Website: https://juliagpu.org/

GitHub Org: https://github.com/JuliaGPU




(We'll focus on Nvidia GPUs but there is [support for other GPUs](https://juliagpu.org/) as well.)

The interface to NVIDIA GPU computing in Julia is [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl).



It provides:

* **High-level abstraction `CuArray`**
* **Tools for writing custom CUDA kernels**
* **Wrappers to proprietary NVIDIA libraries (e.g. CUBLAS, CUFFT, CUSPARSE)**

In [ ]:
using CUDA

In [ ]:
CUDA.versioninfo() # automatically downloads CUDA framework if necessary

In [ ]:
CUDA.functional()

## High-level abstraction: `CuArray`

### GPU memory

A `CuArray` is a CPU handle to GPU memory.

In [ ]:
x_gpu = CuArray{Float32}(undef, 3)

In [ ]:
CUDA.rand(3) # Note: defaults to Float32

In [ ]:
CUDA.zeros(3) # Note: defaults to Float32

 We can readily move data to the GPU by converting to `CuArray`.

 <img src="../imgs/cpu_gpu_transfer.svg" width=180px>

In [ ]:
x_cpu = [1,2,3]
x_gpu = CuArray(x_cpu) 

(or by using `copyto!` to move it into already allocated memory)

#### Memory management

`CuArray`s are managed by Julia's **garbage collector**. In principle, if they are unreachable, they get cleaned up automatically.

However, by default CUDA.jl uses a **memory pool** to speed up allocations. So it might appear as if the objects have not been free'd. (You can disable the pool with `JULIA_CUDA_MEMORY_POOL=none`.)

In [ ]:
CUDA.memory_status()

In [ ]:
x_gpu = CUDA.rand(10_000_000);

In [ ]:
Base.format_bytes(sizeof(x_gpu))

In [ ]:
CUDA.memory_status()

In [ ]:
x_gpu = nothing; GC.gc(true)

In [ ]:
CUDA.memory_status()

We can use `CUDA.unsafe_free!(x_gpu)` and `CUDA.reclaim()` to more aggressively suggest the freeing of the memory.

In [ ]:
#CUDA.unsafe_free!(x_gpu)

In [ ]:
#CUDA.reclaim()

### GPU computation

In [ ]:
CuArray <: AbstractArray

Therefore, we should be able to do all kind of operations with it, that we'd also do with regular `Array`s. (**duck typing**)

#### Example: Matrix multiplication

In [ ]:
N = 1000
A_gpu = CUDA.rand(N,N)
B_gpu = CUDA.rand(N,N)

In [ ]:
A_gpu * B_gpu

In [ ]:
using BenchmarkTools

@btime A_cpu * B_cpu setup=(A_cpu = rand(Float32, N,N); B_cpu = rand(Float32, N,N););
@btime A_gpu * B_gpu setup=(A_gpu = CUDA.rand(N,N); B_gpu = CUDA.rand(N,N););

Note how the timescales change: **milliseconds -> microseconds**!

(`*` for `CuArray`s uses a cuBLAS kernel under the hood)

#### Example: Broadcasting, `map`, `reduce`, etc.

In [ ]:
A_gpu .+ B_gpu # runs on the GPU!

In [ ]:
sqrt.(A_gpu.^2 + B_gpu.^2) # runs on the GPU!

In [ ]:
mapreduce(sin, +, A_gpu) # runs on the GPU!

#### "Counter-example:" Scalar indexing

In [ ]:
A_gpu[1]

In [ ]:
CUDA.@allowscalar A_gpu[1]

In [ ]:
function gpu_not_actually!(C, A, B)
    CUDA.@sync CUDA.@allowscalar for i in eachindex(A,B)
        C[i] = A[i] * B[i] # multiplication will happen on CPU!
    end
end

function gpu_broadcasting!(C, A, B)
    CUDA.@sync C .= A .* B
end

In [ ]:
using BenchmarkTools

N = 10
@btime gpu_not_actually!(C, A, B) setup=(A = CUDA.rand(10,10); B = CUDA.rand(10,10); C = CUDA.rand(10,10););
@btime gpu_broadcasting!(C, A, B) setup=(A = CUDA.rand(10,10); B = CUDA.rand(10,10); C = CUDA.rand(10,10););

##### FLoops: CUDA executor

In [ ]:
using FLoops, FoldsCUDA

function gpu_floops!(C, A, B)
    CUDA.@sync @floop CUDAEx() for i in eachindex(A,B,C)
        C[i] = A[i] * B[i]
    end
end

In [ ]:
@btime gpu_floops!(C, A, B) setup=(A = CUDA.rand(10,10); B = CUDA.rand(10,10); C = CUDA.rand(10,10););

## Kernel programming: Writing CUDA kernels

A CUDA kernel is a function that will be executed by all GPU *threads* separately. Based on the index of a thread we can make them operate on different pieces of given data (Single Program Multiple Data (SPMD) programming model similar to MPI).

In [ ]:
x = CUDA.zeros(1024)

function cuda_kernel!(x)
    i = threadIdx().x
    x[i] += 1
    return nothing # CUDA kernels should never return anything
end

In [ ]:
CUDA.@sync @cuda threads=length(x) cuda_kernel!(x)

In [ ]:
x

Kernel programming can quickly become (much) more difficult though because
* you need to respect **hardware limitations** of the GPU
* **not all operations can readily be expressed as scalar kernels** (e.g. reductions)
* since kernels execute on the GPU, the Julia runtime isn't available and kernel code has limitations (**you can't just write arbitrary Julia code in kernels**)
  * no GC / no allocations
  * must be fully type inferred
  * no `try ... catch ... end`
  * no strings
  * ...

Simple example for a hardware limitation: **A100 supports a maximal number of 1024 threads.**

In [ ]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)

In [ ]:
x = CUDA.zeros(1025)
CUDA.@sync @cuda threads=length(x) cuda_kernel!(x)

**But what if we want to go larger?**

### CUDA programming model

<img src="../imgs/cuda_blocks_threads.png" width=700px>

(Note: in Julia indices start at 1)

**Source:** [Sivalingam, Karthee. "GPU Acceleration of a Theoretical Particle Physics Application." Master's Thesis, The University of Edinburgh (2010).](https://static.epcc.ed.ac.uk/dissertations/hpc-msc/2009-2010/Karthee%20Sivalingam.pdf)

* **Threads** → CUDA cores
* **Blocks** of threads → SMs
* **Grid** of blocks → entire GPU

In [ ]:
function cuda_kernel_blocks!(x)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    if i <= length(x)
        @inbounds x[i] += 1
    end
    return nothing # CUDA kernels should never return anything
end

In [ ]:
x = CUDA.zeros(1025)
CUDA.@sync @cuda threads=1024 blocks=2 cuda_kernel_blocks!(x)

In [ ]:
x

How does our custom CUDA kernel compare to broadcasting?

In [ ]:
function add_one_kernel(x)
    CUDA.@sync @cuda threads=1024 blocks=1 cuda_kernel_blocks!(x)
end

function add_one_broadcasting(x)
    CUDA.@sync x .+ 1
end

@btime add_one_kernel(x) setup=(x = CUDA.zeros(1024););
@btime add_one_broadcasting(x) setup=(x = CUDA.zeros(1024););

### Simplifying kernel launches: [Occupancy API](https://developer.nvidia.com/blog/cuda-pro-tip-occupancy-api-simplifies-launch-configuration/)

Hardcoding limits (1024 above) is rarely a good idea. Also, in reality, the actual maximal number of threads can depend on kernel details, like how many resources the kernel is using.

**The occupancy API is an automatic tool that can be used to obtain good launch parameters.**

In [ ]:
kernel = @cuda launch=false cuda_kernel_blocks!(x)

In [ ]:
CUDA.maxthreads(kernel)

In [ ]:
config = CUDA.launch_configuration(kernel.fun)

Here, the number `blocks` indicates how many blocks we would need to fully occupy the GPU. For a given input `x`, we might need fewer or more blocks.

In [ ]:
threads = min(length(x), config.threads)

In [ ]:
blocks = cld(length(x), threads)

Launching the kernel with the dynamic launch parameters:

In [ ]:
kernel(x; threads, blocks)

## Case study: Three ways to SAXPY on the GPU

**SAXPY** = **S**ingle precision **A** times **X** **P**lus **Y**

In [ ]:
"Computes the SAXPY on the CPU using broadcasting"
function saxpy_broadcast_cpu!(a, x, y)
    y .= a .* x .+ y
end

In [ ]:
"Computes the SAXPY on the GPU using broadcasting"
function saxpy_broadcast_gpu!(a, x, y)
    CUDA.@sync y .= a .* x .+ y
end

In [ ]:
"CUDA kernel for computing the SAXPY on the GPU"
function _saxpy_kernel!(a, x, y)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    if i <= length(y)
        @inbounds y[i] = a * x[i] + y[i]
    end
    return nothing
end

"Computes the SAXPY on the GPU using the custom CUDA kernel `_saxpy_kernel!`"
function saxpy_cuda_kernel!(a, x, y; nthreads, nblocks)
    CUDA.@sync @cuda(
        threads = nthreads,
        blocks = nblocks,
        _saxpy_kernel!(a, x, y)
    )
end

In [ ]:
function saxpy_cublas!(a, x, y)
    CUDA.@sync CUBLAS.axpy!(length(x), a, x, y)
end

In [ ]:
using PrettyTables

"Computes the GFLOP/s from the vector length `len` and the measured runtime `t`."
saxpy_flops(t; len) = 2.0 * len * 1e-9 / t # GFLOP/s

"Computes the GB/s from the vector length `len`, the vector element type `dtype`, and the measured runtime `t`."
saxpy_bandwidth(t; dtype, len) = 3.0 * sizeof(dtype) * len * 1e-9 / t # GB/s

function main()
    if !contains(lowercase(name(device())), "a100")
        @warn("This script was tuned for a NVIDIA A100 GPU. Your GPU: $(name(device())).")
    end
    dtype = Float32
    nthreads = 1024 # CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK
    nblocks = 500_000
    len = nthreads * nblocks # vector length
    a = convert(dtype, 3.1415)
    x = ones(dtype, len)
    y = ones(dtype, len)
    xgpu = CUDA.ones(dtype, len)
    ygpu = CUDA.ones(dtype, len)

    t_broadcast_cpu = @belapsed saxpy_broadcast_cpu!($a, $x, $y) samples = 10 evals = 2
    t_broadcast_gpu = @belapsed saxpy_broadcast_gpu!($a, $xgpu, $ygpu) samples = 10 evals = 2
    t_cuda_kernel = @belapsed saxpy_cuda_kernel!($a, $xgpu, $ygpu; nthreads=$nthreads, nblocks=$nblocks) samples = 10 evals = 2
    t_cublas = @belapsed saxpy_cublas!($a, $xgpu, $ygpu) samples = 10 evals = 2
    times = [t_broadcast_cpu, t_broadcast_gpu, t_cuda_kernel, t_cublas]

    flops = saxpy_flops.(times; len)
    bandwidths = saxpy_bandwidth.(times; dtype, len)

    labels = ["Broadcast (CPU)", "Broadcast (GPU)", "CUDA kernel", "CUBLAS"]
    data = hcat(labels, 1e3 .* times, flops, bandwidths)
    pretty_table(data; header=(["Variant", "Runtime", "FLOPS", "Bandwidth"], ["", "ms", "GFLOP/s", "GB/s"]))
    println("Theoretical Memory Bandwidth: 1555 GB/s")
    return nothing
end

In [ ]:
main()

<img src="../imgs/a100_saxpy_results.png" width=1000px>

## Remarks

* [KernelAbstractions.jl](https://github.com/JuliaGPU/KernelAbstractions.jl): Writing hardware agnostic computational kernels.
* [Tullio.jl](https://github.com/mcabbott/Tullio.jl): Also supports NVIDIA GPUs and can produce more efficient kernels than simple broadcasting.

More on GPU computing in Julia? See e.g. https://www.youtube.com/watch?v=Hz9IMJuW5hU